In [4]:
import spacy
from spacytei.tokenlist import doc_to_tokenlist_no_sents
from spacytei.ner import fetch_ner_samples

In [5]:
nlp = spacy.load(r"C:\Users\pandorfer\Documents\Redmine\rita_project\vfbr\data\extract_persons")

In [8]:
vfbentries = VfbEntry.objects.all()

In [13]:
for x in vfbentries:
    enriched = nlp(x.vollregest)
    resp_dict = fetch_ner_samples(nlp, enriched, dont_split=True)
    for y in resp_dict:
        ner, _ = NerSample.objects.get_or_create(
                                text=y[0]
                            )
        ner.entity_json = y[1]
        ner.content_object = x
        ner.save()

In [15]:
for x in NerSample.objects.all():
    ners = x.return_ents()
    if len(ners) == 0:
        pass
    else:
        for ent in ners:
            try:
                pers = Person.objects.create(written_name=ent)
                x.content_object.mentioned_person.add(pers)
            except Exception as e:
                print("Error for ent: {} in VfbEntry-ID: {}. Error: {}".format(ent, x.id, e))

In [56]:
to_delete = [
    'Verlassenschaf',
    'Testa',
    'Bestandskontrakt',
    'Bericht',
    'Abrechnung',
    'Anhang',
    'Anweiserschaftsrechnun',
    'Besitzübergabe'
    'Gerhabschaftsrechnu',
    'Geburtsurkunde',
    'rechnung'
    'überlassungsv',
    'kontrakt',
    'Protokoll',
    'iquidatio',
    'Klage („obri',
    'Quittung',
    'estätigung',
    'vertrag',
    'inventar',
    'Erbteil',
    'schuldbr',
    'verinbar',
    'vermögen',
    'vollmach',
    'wiedergut',
    '. Dazu',
    'abschrift',
    'vereinbarung',
    'Versorg',
    'Schuldenüb',
    '] Besitzerlassung bzw. Besitzab'
    '– Da',
]

In [57]:
for x in to_delete:
    Person.objects.filter(written_name__icontains=x).delete()

In [58]:
### enrich Persons; extract VN, NN

In [18]:
# NerSample.objects.all().delete()

(275, {'annotations.NerSample': 275})

In [19]:
nlp = spacy.load(r"C:\Users\pandorfer\Documents\Redmine\rita_project\vfbr\data\vfbr-names-model")

In [20]:
for x in Person.objects.all():
    enriched = nlp(x.written_name)
    resp_dict = fetch_ner_samples(nlp, enriched, dont_split=True, ent_types=['VN', 'NN'])
    for y in resp_dict:
        ner, _ = NerSample.objects.get_or_create(
                                text=y[0]
                            )
        ner.entity_json = y[1]
        ner.content_object = x
        ner.save()